# 4.0.3 Sampling Around Event Boundaries

An approach to handling the variability in videos lengths is to sample clips around event boundaries and show the events there.

## Jupyter Extensions

Load [watermark](https://github.com/rasbt/watermark) to see the state of the machine and environment that's running the notebook. To make sense of the options, take a look at the [usage](https://github.com/rasbt/watermark#usage) section of the readme.

In [1]:
# Load `watermark` extension
%load_ext watermark
# Display the status of the machine and packages. Add more as necessary.
%watermark -v -n -m -g -b -t -p torch,torchvision,cv2,h5py,pandas,matplotlib,seaborn,jupyterlab,lab

Mon Feb 24 2020 10:46:45 

CPython 3.6.10
IPython 7.12.0

torch 1.2.0
torchvision 0.1.8
cv2 3.4.2
h5py 2.8.0
pandas 1.0.1
matplotlib 3.1.3
seaborn 0.10.0
jupyterlab 1.2.6
lab 0+untagged.21.g9d80dae.dirty

compiler   : GCC 7.3.0
system     : Linux
release    : 4.4.0-173-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 16
interpreter: 64bit
Git hash   : 9d80daec6156826fef6a7472507201013d8c7801
Git branch : master


Load [autoreload](https://ipython.org/ipython-doc/3/config/extensions/autoreload.html) which will always reload modules marked with `%aimport`.

This behavior can be inverted by running `autoreload 2` which will set everything to be auto-reloaded *except* for modules marked with `%aimport`.

In [2]:
# Load `autoreload` extension
%load_ext autoreload
# Set autoreload behavior
%autoreload 1

Load `matplotlib` in one of the more `jupyter`-friendly [rich-output modes](https://ipython.readthedocs.io/en/stable/interactive/plotting.html). Some options (that may or may not have worked) are `inline`, `notebook`, and `gtk`.

In [3]:
# Set the matplotlib mode
%matplotlib inline

## Set the GPU

Make sure we aren't greedy.

In [4]:
!nvidia-smi

Mon Feb 24 10:47:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)    Off  | 00000000:04:00.0 Off |                  N/A |
| 23%   22C    P8     7W / 250W |      0MiB / 12196MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN X (Pascal)    Off  | 00000000:05:00.0 Off |                  N/A |
| 29%   50C    P2    58W / 250W |   2885MiB / 12196MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [5]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


## Imports

In [6]:
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm

Local imports that may or may not be autoreloaded. This section contains things that will likely have to be re-imported multiple times, and have additions or subtractions made throughout the project.

In [7]:
# Constants to be used throughout the package
%aimport lab.index
from lab.index import DIR_DATA_INT, DIR_DATA_RAW
%aimport lab.breakfast.constants
from lab.breakfast.constants import SEED
# Import the data subdirectories
%aimport lab.breakfast.index
from lab.breakfast.index import (DIR_BREAKFAST, 
                                 DIR_BREAKFAST_DATA, 
                                 DIR_COARSE_SEG, 
                                 DIR_FINE_SEG,
                                )

## Initial Setup

Set [seaborn defaults](https://seaborn.pydata.org/generated/seaborn.set.html) for matplotlib.

In [8]:
sns.set()

## Rerunning the Same Code Blocks

### Getting Start Frames and Number of Segments

In [9]:
df = pd.read_csv(str(DIR_BREAKFAST / 'video_lengths.csv'),
                 index_col=0,
                 header=0)

In [10]:
df.head()

,Name,Length
0,P27_cam01_P27_scrambledegg,3561
1,P11_cam01_P11_scrambledegg,2649
2,P27_stereo01_P27_scrambledegg,3560
3,P21_cam01_P21_scrambledegg,5281
4,P46_cam02_P46_scrambledegg,3135


In [11]:
with open(str(DIR_COARSE_SEG / 'pancake/P53_webcam01_P53_pancake.txt'), 'r') as f:
    for l in f:
        print(l)

1-1 SIL  

2-302 spoon_flour  

303-501 crack_egg  

502-727 pour_milk  

728-1077 butter_pan  

1078-1844 stir_dough  

1845-1948 pour_dough2pan  

1949-3649 fry_pancake  

3650-3736 take_plate  

3737-3826 put_pancake2plate  

3827-3874 SIL  



In [12]:
with open(str(DIR_COARSE_SEG / 'pancake/P53_webcam01_P53_pancake.txt'), 'r') as f:
    starts = [line.split('-')[0] for line in f][1:]
    print(starts)

['2', '303', '502', '728', '1078', '1845', '1949', '3650', '3737', '3827']


In [13]:
n_coarse_segments, coarse_segment_starts = [], []

for name in df.Name:
    action = name.split('_')[-1]
    path = DIR_COARSE_SEG / f'{action}/{name}.txt'
    
    with open(str(path), 'r') as file:
        coarse_segment_starts.append([int(line.split('-')[0]) for line in file][1:])
        n_coarse_segments.append(len(coarse_segment_starts[-1]))

In [14]:
df['n_coarse_segments'] = n_coarse_segments
df['coarse_segment_starts'] = coarse_segment_starts

In [15]:
df.head()

,Name,Length,n_coarse_segments,coarse_segment_starts
0,P27_cam01_P27_scrambledegg,3561,9,"[12, 185, 444, 1031, 1310, 1923, 2880, 2987, 3..."
1,P11_cam01_P11_scrambledegg,2649,6,"[39, 293, 897, 1928, 2410, 2620]"
2,P27_stereo01_P27_scrambledegg,3560,9,"[12, 185, 444, 1031, 1310, 1923, 2880, 2987, 3..."
3,P21_cam01_P21_scrambledegg,5281,9,"[234, 624, 1120, 1408, 1577, 1975, 4189, 4451,..."
4,P46_cam02_P46_scrambledegg,3135,7,"[2, 272, 754, 1226, 1421, 2617, 2682]"


In [16]:
n_fine_segments, fine_segment_starts = [], []

for name in df.Name:
    action = name.split('_')[-1]
    path = DIR_FINE_SEG / f'{action}/{name}.txt'
    
    try:
        with open(str(path), 'r') as file:
            fine_segment_starts.append([int(line.split('-')[0]) for line in file][1:-1])
            n_fine_segments.append(len(fine_segment_starts[-1]))
    except FileNotFoundError:
        fine_segment_starts.append(None)
        n_fine_segments.append(None)
        
df['n_fine_segments'] = n_fine_segments
df['fine_segment_starts'] = fine_segment_starts

df.tail()

,Name,Length,n_coarse_segments,coarse_segment_starts,n_fine_segments,fine_segment_starts
1024,P36_webcam02_P36_pancake,7049,8,"[2, 279, 548, 1044, 1958, 2243, 2844, 6903]",NaN,None
1025,P54_stereo01_P54_pancake,3340,11,"[2, 225, 506, 692, 1018, 1336, 1709, 1837, 206...",66.0,"[11, 27, 55, 77, 115, 132, 191, 223, 235, 251,..."
1026,P13_cam01_P13_pancake,4436,12,"[31, 611, 717, 907, 1036, 1316, 1443, 1493, 17...",111.0,"[32, 47, 93, 138, 212, 318, 333, 348, 377, 467..."
1027,P21_cam01_P21_pancake,5841,10,"[95, 415, 699, 960, 1634, 2125, 2437, 5367, 54...",NaN,None
1028,P12_webcam01_P12_pancake,2870,8,"[2, 136, 324, 446, 603, 744, 2680, 2805]",54.0,"[32, 47, 62, 76, 77, 122, 123, 152, 213, 287, ..."


In [17]:
len([n for n in n_fine_segments if n is None])

505

In [18]:
len(n_fine_segments)

1029

### Counting Actions

In [19]:
actions = []
for name in df.Name:
    actions.append(name.split('_')[-1])
df['actions'] = actions

In [20]:
df.head()

,Name,Length,n_coarse_segments,coarse_segment_starts,n_fine_segments,fine_segment_starts,actions
0,P27_cam01_P27_scrambledegg,3561,9,"[12, 185, 444, 1031, 1310, 1923, 2880, 2987, 3...",71.0,"[38, 73, 83, 150, 183, 213, 259, 308, 314, 337...",scrambledegg
1,P11_cam01_P11_scrambledegg,2649,6,"[39, 293, 897, 1928, 2410, 2620]",74.0,"[23, 34, 56, 67, 87, 107, 116, 122, 128, 132, ...",scrambledegg
2,P27_stereo01_P27_scrambledegg,3560,9,"[12, 185, 444, 1031, 1310, 1923, 2880, 2987, 3...",71.0,"[38, 73, 83, 150, 183, 213, 259, 308, 314, 337...",scrambledegg
3,P21_cam01_P21_scrambledegg,5281,9,"[234, 624, 1120, 1408, 1577, 1975, 4189, 4451,...",NaN,None,scrambledegg
4,P46_cam02_P46_scrambledegg,3135,7,"[2, 272, 754, 1226, 1421, 2617, 2682]",NaN,None,scrambledegg


In [21]:
np.unique(df.actions, return_counts=True)

(array(['coffee', 'friedegg', 'juice', 'milk', 'pancake', 'sandwich',
        'scrambledegg'], dtype=object),
 array([167, 173, 162, 187, 157, 169,  14]))

In [22]:
df.tail()

,Name,Length,n_coarse_segments,coarse_segment_starts,n_fine_segments,fine_segment_starts,actions
1024,P36_webcam02_P36_pancake,7049,8,"[2, 279, 548, 1044, 1958, 2243, 2844, 6903]",NaN,None,pancake
1025,P54_stereo01_P54_pancake,3340,11,"[2, 225, 506, 692, 1018, 1336, 1709, 1837, 206...",66.0,"[11, 27, 55, 77, 115, 132, 191, 223, 235, 251,...",pancake
1026,P13_cam01_P13_pancake,4436,12,"[31, 611, 717, 907, 1036, 1316, 1443, 1493, 17...",111.0,"[32, 47, 93, 138, 212, 318, 333, 348, 377, 467...",pancake
1027,P21_cam01_P21_pancake,5841,10,"[95, 415, 699, 960, 1634, 2125, 2437, 5367, 54...",NaN,None,pancake
1028,P12_webcam01_P12_pancake,2870,8,"[2, 136, 324, 446, 603, 744, 2680, 2805]",54.0,"[32, 47, 62, 76, 77, 122, 123, 152, 213, 287, ...",pancake


### Finding Good First Frames

In [23]:
# Select this number of frames
n_frames = 64

In [24]:
np.unique(n_coarse_segments, return_counts=True)

(array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 17]),
 array([ 41,  71, 225, 243, 151,  88,  48,  33,  50,  50,  17,   6,   2,
          4]))

In [25]:
starting_segments = [coarse_segment_starts[i][0]
                     for i, n in enumerate(n_coarse_segments)
                     if n < 3]

In [26]:
np.unique(starting_segments, return_counts=True)

(array([ 2, 25, 38, 47, 60, 72]), array([20,  5,  5,  3,  4,  4]))

Should ignore indices that start with 2

## Making the List of Indices

Each element of the list corresponds to a row in the dataframe.

In [79]:
np.random.seed(SEED)

all_clip_ranges = []
all_clip_diffs = []
start_clip_idcs = []

buffer = 15 # Assume people correctly label within a second of event

for n, starts, length in zip(df.n_coarse_segments, df.coarse_segment_starts, df.Length):
    # Ignore last segment which is start of SIL
    idcs_to_use = starts[:-1]
    # 2 Shows up disproportionally more than others. Ignore it
    if 2 in idcs_to_use:
        idcs_to_use = idcs_to_use[1:]
        
    # Move on if there are no events left to use
    if not idcs_to_use:
        all_clip_ranges.append([])
        event_clip_idcs.append(np.array([]))
        start_clip_idcs.append([])
        continue 
    
    clip_starts, clip_ranges = [], []
    
    # Calculate differences between each of the start points
    diffs = np.diff(idcs_to_use)
    for i, diff in enumerate(diffs):
        # If the value is greater than n_frames + 2*buffer,
        if diff > n_frames + 2*buffer:
            # Decide how many clips to create based on how many times larger
            n_clips = (diff - 2*buffer) // n_frames
            
            # use that starting point + buffer
            diff_clip_starts = [idcs_to_use[i] + 2*buffer + clip*n_frames
                                for clip in range(n_clips)]
            # Build the ranges
            clip_ranges += [np.arange(start, start+n_frames)
                            for start in diff_clip_starts]
            clip_starts += diff_clip_starts
    
    # List of the full 64 frame clip indices
    all_clip_ranges.append(clip_ranges)
    # And the starting idx of the clip
    start_clip_idcs.append(clip_starts)
    # All the diffs
    all_clip_diffs.append(diffs)
    
print(n_frames, buffer)
# print(idcs_to_use)
# print(diffs)
# print(n_clips)
# print(len(clip_starts))
# print(len(clip_ranges))
print(len(all_clip_ranges))
print(len(start_clip_idcs))

64 15
1029
1029


### Saving the Interim Data

In [54]:
path_interim_data = DIR_DATA_INT / 'breakfast/video_clips/nonevent_clips'
if not path_interim_data.exists():
    path_interim_data.mkdir(parents=True)
    
with open(str(path_interim_data / f'nonevent_clip_indices_seed_{SEED}.pkl'), 'wb') as filehandle:
    pickle.dump(event_clip_idcs, filehandle)

In [55]:
with open(str(path_interim_data / f'nonevent_clip_ranges_seed_{SEED}.pkl'), 'wb') as filehandle:
    pickle.dump(all_clip_ranges, filehandle)

In [56]:
with open(str(path_interim_data / f'nonevent_clip_starts_seed_{SEED}.pkl'), 'wb') as filehandle:
    pickle.dump(start_clip_idcs, filehandle)

## Creating the 64 Dim FV Clips

In [57]:
path_clips = path_interim_data / '64dim_fv_clips'
if not path_clips.exists():
    path_clips.mkdir(parents=True)

In [60]:
for name, action, clip_ranges, starts, length in zip(df.Name, 
                                                     df.actions,
                                                     all_clip_ranges,
                                                     start_clip_idcs,
                                                     df.Length):
    path_file = DIR_BREAKFAST_DATA / f'{action}/{name}.txt'
    assert path_file.exists(), path_file
    
    file_array = pd.read_table(str(path_file), index_col=0, header=0).to_numpy()
    assert file_array.shape[0] == length
    
    for clip_range, start in zip(clip_ranges, starts):
        path_out = path_clips / f'{name}_start_{start}_seed_{SEED}.npy'
        np.save(str(path_out), file_array[clip_range, :])

IndexError: index 1186 is out of bounds for axis 0 with size 1186

In [62]:
clip_range

array([1156, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1164, 1165, 1166,
       1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177,
       1178, 1179, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1187, 1188,
       1189, 1190, 1191, 1192, 1193, 1194, 1195, 1196, 1197, 1198, 1199,
       1200, 1201, 1202, 1203, 1204, 1205, 1206, 1207, 1208, 1209, 1210,
       1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219])

In [63]:
length

1186

In [68]:
idx = int(df.index[df.Name == name].values)
idx

259

In [80]:
all_clip_diffs[idx]

array([ 185,  210, 1700,  270])

In [81]:
start_clip_idcs[idx]

[185,
 249,
 370,
 434,
 580,
 644,
 708,
 772,
 836,
 900,
 964,
 1028,
 1092,
 1156,
 1220,
 1284,
 1348,
 1412,
 1476,
 1540,
 1604,
 1668,
 1732,
 1796,
 1860,
 1924,
 1988,
 2052,
 2116,
 2180,
 2280,
 2344,
 2408]

In [84]:
coarse_segment_starts[idx]

[155, 340, 550, 2250, 2520, 2555]

In [82]:
df.Length[idx]

1186

In [90]:
df[df.Name == name]

,Name,Length,n_coarse_segments,coarse_segment_starts,n_fine_segments,fine_segment_starts,actions
259,P52_stereo01_P52_sandwich,1186,6,"[155, 340, 550, 2250, 2520, 2555]",73.0,"[358, 507, 517, 542, 595, 619, 683, 701, 716, ...",sandwich


In [91]:
test_file = pd.read_table(str(DIR_BREAKFAST_DATA / f'{action}/{name}.txt'), index_col=0, header=0).to_numpy()

In [92]:
test_file.shape

(1186, 64)

## Creating One Dataset

In [239]:
all_clips_data = [[path, np.load(str(path))] for path in path_clips.iterdir()]

In [252]:
path_all_clips, all_data = zip(*all_clips_data)

In [251]:
path_all_clips[:5]

(PosixPath('/media/data_cifs/apra/work/labwork/data/interim/breakfast/event_clips/64dim_fv_clips/P54_webcam02_P54_milk_start_174_event_42_seed_117.npy'),
 PosixPath('/media/data_cifs/apra/work/labwork/data/interim/breakfast/event_clips/64dim_fv_clips/P12_cam01_P12_sandwich_start_674_event_41_seed_117.npy'),
 PosixPath('/media/data_cifs/apra/work/labwork/data/interim/breakfast/event_clips/64dim_fv_clips/P38_webcam02_P38_friedegg_start_532_event_57_seed_117.npy'),
 PosixPath('/media/data_cifs/apra/work/labwork/data/interim/breakfast/event_clips/64dim_fv_clips/P53_webcam02_P53_friedegg_start_960_event_54_seed_117.npy'),
 PosixPath('/media/data_cifs/apra/work/labwork/data/interim/breakfast/event_clips/64dim_fv_clips/P49_cam01_P49_pancake_start_5180_event_61_seed_117.npy'))

In [257]:
path_raw_64fv_data = DIR_DATA_RAW / 'breakfast/64dim_fv_clips'
if not path_raw_64fv_data.exists():
    path_raw_64fv_data.mkdir(parents=True)

In [258]:
with open(str(path_raw_64fv_data / f'path_all_data_seed_{SEED}.pkl'), 'wb') as filehandle:
    pickle.dump(path_all_clips, filehandle)

In [248]:
all_data_array = np.array(all_data)
all_data_array.shape

(4794, 64, 64)

In [261]:
np.save(str(path_raw_64fv_data / f'all_clips_seed_{SEED}.npy'), all_data_array)